In [ ]:
import pandas as pd
import numpy  as np
import joblib
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("data/EA_CSV.csv", engine="python", encoding="latin1")

In [ ]:
df.head()

In [ ]:
#turn the first row into column
def get_data(df):
    #combine the description
    df['description'] = df['Expertise'].fillna('') + " " + df['Title'].fillna('')
    return df

In [ ]:
def clean_text(s):
    #removeNonAscii
    s = "".join(i for i in s if ord(i)<128)

    #return all lower cases
    s = s.lower()

    #remove stop wrods
    s = s.split()
    stops = set(stopwords.words("english"))
    text = [w for w in s if not w in stops]
    text = " ".join(text)

    #remove html
    html_pattern = re.compile('<.*?>')
    text = html_pattern.sub(r'', text)

    #remove punctuation
    text = re.sub(r'[^\w\s]'," ",text)
    return text



In [ ]:
def glove_model(file_path='glove.twitter.27B.200d.txt'):
  """
  input: the access to the file which contains the pre-trained glove model
  ouput: a trained model using glove
  """

  embeddings_index = {}
  f = open(file_path, encoding='utf-8')
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()
  return embeddings_index

In [ ]:
def glove2vec(text, model):
    """
    input: 
    - text: the string that you wanted to turn into vectors
    - model: the glove pre-trained model that you wanted to use

    output:
    an average vector of your input text using the glove model

    """
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    vectors = {}

    avgword2vec = None
    count = 0

    for word in text.split():
        if word in model:
            count += 1
            if avgword2vec is None:
                avgword2vec = model[word]
            else:
                avgword2vec = avgword2vec + model[word]
            
    if avgword2vec is not None:
        avgword2vec = avgword2vec / count
    
    word_embeddings.append(avgword2vec)

    return word_embeddings

In [ ]:
def recommendations_glove(input, df=df, col='description'):
    """
    input: 
    - input: key words relating to the article
    - df: the dataframe that we are using
    - col: the column name that contains description of EAs
    - file path: the file path to the pre-trained glove model

    output:
    a list of EAs' names ranked by the most recommended / most suitable EA
    to the least
    """

    #get the model (it is too slow to compute the model every time)
    model = glove_model(file_path='glove.twitter.27B.200d.txt')

    # vecterizd the input
    vector_input = glove2vec(input, model)

    # vecterized all the descriptions of EAs
    df[col] = df[col].apply(clean_text)
    EAs_vectors = [glove2vec(EA, model) for EA in df[col]]

    #drop an EA if that EA has "None" vector
    df['vectors'] = EAs_vectors
    for i, n in enumerate(df['vectors']):
      if n[0] is None:
        df = df.drop([i+1], axis=0)


    # finding cosine similarity for the vectors
    similarity = []
    for n in df['vectors']:
      scores = cosine_similarity(vector_input, n)[0][0]
      similarity.append(scores)
    
    df['similarity'] = similarity


    # sort and find the recommended movie
    df = df.sort_values(by=['similarity'], ascending=False)
    #res_df = df.iloc[:m]


    return df['Names']

In [ ]:
def input_vec(input, model):
    avgword2vec = None
    count = 0
    for word in input.split():
        if word in model:
            count += 1
            if avgword2vec is None:
                avgword2vec = model[word]
            else:
                avgword2vec = avgword2vec + model[word]

    if avgword2vec is not None:
        avgword2vec = avgword2vec / count

    return avgword2vec

In [ ]:
input_vec(keywords, model)

In [ ]:
def recomm_engine(keywords, vec_df, model):
    """
    assummed we directly import vecterized EA information and trained model.
    """
    # vectorize the input
    cleaned_input = clean_text(keywords)
    input_vector = input_vec(cleaned_input, model)

    # finding cosine similarity for the vectors
    similarity = []
    input_vector = input_vector.reshape(1,-1)
    for n in vec_df['vectors']:
        n = n.reshape(1,-1)
        scores = cosine_similarity(input_vector, n)[0][0]
        similarity.append(scores)

    vec_df['similarity'] = similarity

    # sort and find the recommended movie
    vec_df = vec_df.sort_values(by=['similarity'], ascending=False)

    return vec_df.loc[::, :'Title']

In [ ]:
vec_df = pd.read_pickle('models/vectors.pkl')

model = joblib.load('models/glove_model.pkl')
keywords = 'data collection, data ethics issues/initial assumptions'

In [ ]:
recomm_engine(keywords, vec_df, model)

In [ ]:
vec_df